In [ ]:
!pip install deap

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from deap import base, creator, tools, algorithms
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv("/content/Patients - Patients.csv")

In [ ]:
medical_data = data.iloc[:, [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 13]].values

In [ ]:
labels = data['Test Results']

In [ ]:
data.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Tiffany Ramirez,81,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,37490.98336,146,Elective,2022-12-01,Aspirin,Inconclusive
1,Ruben Burns,35,Male,O+,Asthma,2023-06-01,Diane Jackson,"Burke, Griffin and Cooper",UnitedHealthcare,47304.06485,404,Emergency,2023-06-15,Lipitor,Normal
2,Chad Byrd,61,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,36874.89700,292,Emergency,2019-02-08,Lipitor,Normal
3,Antonio Frederick,49,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,23303.32209,480,Urgent,2020-05-03,Penicillin,Abnormal
4,Mrs. Brandy Flowers,51,Male,O-,Arthritis,2021-07-09,Dustin Griffin,"Jones, Brown and Murray",UnitedHealthcare,18086.34418,477,Urgent,2021-08-02,Paracetamol,Normal


In [ ]:
gender = LabelEncoder()
data.iloc[:, 2] = gender.fit_transform(data.iloc[:, 2])

blood = LabelEncoder()
data.iloc[:, 3] = blood.fit_transform(data.iloc[:, 3])

med = LabelEncoder()
data.iloc[:, 4] = med.fit_transform(data.iloc[:, 4])

dr = LabelEncoder()
data.iloc[:, 6] = dr.fit_transform(data.iloc[:, 6])

hosp = LabelEncoder()
data.iloc[:, 7] = hosp.fit_transform(data.iloc[:, 7])

insu = LabelEncoder()
data.iloc[:, 8] = insu.fit_transform(data.iloc[:, 8])

adm = LabelEncoder()
data.iloc[:, 11] = adm.fit_transform(data.iloc[:, 11])

medication = LabelEncoder()
data.iloc[:, 13] = medication.fit_transform(data.iloc[:, 13])

<ipython-input-11-5d84b93da5d4>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.iloc[:, 2] = gender.fit_transform(data.iloc[:, 2])
<ipython-input-11-5d84b93da5d4>:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.iloc[:, 3] = blood.fit_transform(data.iloc[:, 3])
<ipython-input-11-5d84b93da5d4>:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-u

In [ ]:
medical_data = data.iloc[:, [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 13]].values

In [ ]:
def evaluate(individual):
    weights = np.array(individual)
    weighted_data = processed_data * weights
    knn = KNeighborsClassifier()
    knn.fit(weighted_data, labels)
    accuracy = knn.score(weighted_data, labels)
    return accuracy,


POPULATION_SIZE = 50
GENERATIONS = 2
CROSSOVER_PROB = 0.9
MUTATION_PROB = 0.1

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_float", np.random.uniform, 0, 1)

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, len(medical_data[0]))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

processed_data = (np.array(medical_data))

population = toolbox.population(n=POPULATION_SIZE)

fitnesses = list(map(toolbox.evaluate, population))
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit

for gen in range(1, GENERATIONS + 1):
    print("Generation ", gen, "\n")
    offspring = algorithms.varAnd(population, toolbox, cxpb=CROSSOVER_PROB, mutpb=MUTATION_PROB)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for ind, fit in zip(offspring, fits):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))

best_individual = tools.selBest(population, k=1)[0]
best_weights = np.array(best_individual)

print("Best attribute weights :\n\n", best_weights)

Generation  1 

Generation  2 

Best attribute weights :

 [ 0.851311    0.30713841  0.65101704  0.52929236  0.92126068  0.86852958
 -0.08599251  0.01130271 -0.10066637  0.62170907  0.68484364]
